In [1]:
# skorch : pytorch를 쉽게 구현할 수 있는 라이브러리
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [2]:
X_train= X_train.reshape(-1,784)
X_test= X_test.reshape(-1,784)
X_train=X_train/255.
X_test=X_test/255.

In [3]:
import torch
X_train=torch.from_numpy(X_train).float()
y_train=torch.from_numpy(y_train.astype('int32')).long()
X_test=torch.from_numpy(X_test).float()
y_test=torch.from_numpy(y_test.astype('int32')).long()
print(X_train.shape)
print(X_test.shape)
X_train=X_train.cuda()
y_train=y_train.cuda()
X_test=X_test.cuda()
y_test=y_test.cuda()

torch.Size([60000, 784])
torch.Size([10000, 784])


In [4]:
import torch.nn as nn
import torch.nn.functional as F
# 신경망 구성
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.softmax(x, dim=-1)
model=Net().cuda()

In [6]:
from skorch import NeuralNetClassifier
net = NeuralNetClassifier(Net,max_epochs=20,lr=0.1)
net.fit(X_train, y_train)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.6365       0.8993        0.3520  1.3123
      2        0.3191       0.9154        0.2931  1.2687
      3        0.2708       0.9258        0.2567  1.4892
      4        0.2347       0.9333        0.2271  1.6182
      5        0.2059       0.9411        0.2031  1.6320
      6        0.1825       0.9466        0.1846  2.0007
      7        0.1637       0.9520        0.1699  1.7471
      8        0.1482       0.9556        0.1579  1.7136
      9        0.1352       0.9583        0.1482  2.6469
     10        0.1241       0.9604        0.1403  2.0546
     11        0.1145       0.9620        0.1335  1.7055
     12        0.1062       0.9638        0.1279  1.6936
     13        0.0989       0.9653        0.1231  1.7141
     14        0.0924       0.9664        0.1189  1.6918
     15        0.0867       0.9668        0.1156  1.6969
     16        0.0815       0.9

<class 'skorch.classifier.NeuralNetClassifier'>[initialized](
  module_=Net(
    (fc1): Linear(in_features=784, out_features=128, bias=True)
    (fc2): Linear(in_features=128, out_features=10, bias=True)
  ),
)

In [7]:
import numpy as np
# 모형의 정확도 계산
pred = net.predict(X_test)
y_test = y_test.cpu().numpy()
accuracy = np.mean(pred == y_test)
accuracy

0.9723

In [8]:
from torchinfo import summary
summary(model, input_size=(100,784))

Layer (type:depth-idx)                   Output Shape              Param #
Net                                      [100, 10]                 --
├─Linear: 1-1                            [100, 128]                100,480
├─Linear: 1-2                            [100, 10]                 1,290
Total params: 101,770
Trainable params: 101,770
Non-trainable params: 0
Total mult-adds (M): 10.18
Input size (MB): 0.31
Forward/backward pass size (MB): 0.11
Params size (MB): 0.41
Estimated Total Size (MB): 0.83